In [3]:
!pip install python-dotenv cython

  Using cached Cython-3.0.11-py2.py3-none-any.whl.metadata (3.2 kB)
Using cached Cython-3.0.11-py2.py3-none-any.whl (1.2 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
%load_ext cython

# Must set the env var before the RESTClient class is loaded because they use a default arg initializer.

In [1]:
%load_ext dotenv
%dotenv -o

import os
not not os.environ["POLYGON_API_KEY"]

True

In [2]:
import pyarrow as pa
import pyarrow.csv as pa_csv
from pyarrow import dataset as pa_ds
from pyarrow import compute as pa_compute
from pyarrow import fs as pa_fs
from pathlib import Path
import pandas as pd
from itables import show
import os
from fsspec.implementations.arrow import ArrowFSWrapper

from zipline_polygon_bundle.config import PolygonConfig

In [3]:
os.environ["POLYGON_DATA_DIR"]

'/media/mirror/files.polygon.io'

In [4]:
config = PolygonConfig(
    environ=os.environ,
    calendar_name="NYSE",
    # start_date="2020-10-07",
    # end_date="2020-10-15",
    # start_date="2003-10-01",
    start_date="2016-01-01",
    # start_date="2023-01-03",
    end_date="2016-01-06",
    # end_date="2022-12-30",
    # end_date="2023-12-28",
    # end_date="2020-12-31",
    # end_date="2024-06-30",
)

In [5]:
fsspec = ArrowFSWrapper(config.filesystem)
# We sort by path because they have the year and month in the dir names and the date in the filename.
paths = sorted(
    fsspec.glob(os.path.join(config.trades_dir, config.csv_paths_pattern))
)
len(paths), paths[:10]

(5353,
 ['/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-10.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-11.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-12.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-15.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-16.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-17.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-18.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-19.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-22.csv.gz',
  '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-23.csv.gz'])

In [6]:
paths = [
    '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-04.csv.gz',
    '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-05.csv.gz',
    '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-06.csv.gz',
]
paths

['/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-04.csv.gz',
 '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-05.csv.gz',
 '/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-06.csv.gz']

In [13]:
# schema = format.inspect(paths[0], filesystem=filesystem)

# Polygon price scale is 4 decimal places (i.e. hundredths of a penny), but we'll use 10 because we have precision to spare.
# price_type = pa.decimal128(precision=38, scale=10)
# 64bit float a little overkill but avoids any plausible truncation error.
price_type = pa.float64()

# Polygon Aggregate flatfile timestamps are in nanoseconds (like trades), not milliseconds as the docs say.
# I make the timestamp timezone-aware because that's how Unix timestamps work and it may help avoid mistakes.
# timestamp_type = pa.timestamp("ns", tz="UTC")

# There is some problem reading the timestamps as timestamps so we have to read as integer then change the schema.
timestamp_type = pa.int64()
# timestamp_type = pa.timestamp("ns", tz="UTC")

raw_schema = pa.schema(
        [
            pa.field("ticker", pa.string(), nullable=False),
            pa.field("conditions", pa.string(), nullable=False),
            pa.field("correction", pa.string(), nullable=False),
            pa.field("exchange", pa.int8(), nullable=False),
            pa.field("id", pa.string(), nullable=False),
            pa.field("participant_timestamp", timestamp_type, nullable=True),
            pa.field("price", price_type, nullable=False),
            pa.field("sequence_number", pa.int64(), nullable=True),
            pa.field("sip_timestamp", timestamp_type, nullable=False),
            pa.field("size", pa.int64(), nullable=False),
            pa.field("tape", pa.int8(), nullable=False),
            pa.field("trf_id", pa.int64(), nullable=False),
            pa.field("trf_timestamp", timestamp_type, nullable=True),
        ]
    )

timestamp_type = pa.timestamp("ns", tz="UTC")
real_schema = pa.schema(
        [
            pa.field("ticker", pa.string(), nullable=False),
            pa.field("conditions", pa.string(), nullable=False),
            pa.field("correction", pa.string(), nullable=False),
            pa.field("exchange", pa.int8(), nullable=False),
            pa.field("id", pa.string(), nullable=False),
            pa.field("participant_timestamp", timestamp_type, nullable=False),
            pa.field("price", price_type, nullable=False),
            pa.field("sequence_number", pa.int64(), nullable=False),
            pa.field("sip_timestamp", timestamp_type, nullable=False),
            pa.field("size", pa.int64(), nullable=False),
            pa.field("tape", pa.int8(), nullable=False),
            pa.field("trf_id", pa.int64(), nullable=False),
            pa.field("trf_timestamp", timestamp_type, nullable=False),
        ]
    )

convert_options = pa_csv.ConvertOptions(timestamp_parsers=[pa.csv.ISO8601])
format = pa_ds.CsvFileFormat(convert_options=convert_options)


trades_ds = pa_ds.FileSystemDataset.from_paths(paths, format=format, schema=raw_schema, filesystem=config.filesystem)
trades_ds

In [14]:
trades_ds.schema

ticker: string not null
conditions: string not null
correction: string not null
exchange: int8 not null
id: string not null
participant_timestamp: int64
price: double not null
sequence_number: int64
sip_timestamp: int64 not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: int64

In [15]:
fragments = trades_ds.get_fragments()
fragment = next(fragments)
fragment

<pyarrow.dataset.FileFragment type=csv path=/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2016/01/2016-01-04.csv.gz>

In [16]:
%%time
table = fragment.to_table(schema=trades_ds.schema)
table = table.cast(real_schema)
table

CPU times: user 15.4 s, sys: 1.01 s, total: 16.5 s
Wall time: 8.88 s


pyarrow.Table
ticker: string not null
conditions: string not null
correction: string not null
exchange: int8 not null
id: string not null
participant_timestamp: timestamp[ns, tz=UTC] not null
price: double not null
sequence_number: int64 not null
sip_timestamp: timestamp[ns, tz=UTC] not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: timestamp[ns, tz=UTC] not null
----
ticker: [["A","A","A","A","A",...,"A","A","A","A","A"],["A","A","A","A","A",...,"A","A","A","A","A"],...,["ZVZZT","ZVZZT","ZVZZT","ZVZZT","ZVZZT",...,"ZYNE","ZYNE","ZYNE","ZYNE","ZYNE"],["ZYNE","ZYNE","ZYNE","ZYNE","ZYNE",...,"ZYNE","ZYNE","ZYNE","ZYNE","ZYNE"]]
conditions: [["12,37","37","16","16","17,37",...,"14","14","14","","14"],["14","14","14","14","",...,"","","","14,37",""],...,["14,37","14,37","14,37","37","14,37",...,"14","","","",""],["","","","","",...,"15","8,9","15","12,37","12,2"]]
correction: [["0","0","0","0","0",...,"0","0","0","0","0"],["0","0","0","0","0",...,"0","0

In [17]:
table = table.cast(real_schema)
df = table.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43663717 entries, 0 to 43663716
Data columns (total 13 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   ticker                 object             
 1   conditions             object             
 2   correction             object             
 3   exchange               int8               
 4   id                     object             
 5   participant_timestamp  datetime64[ns, UTC]
 6   price                  float64            
 7   sequence_number        int64              
 8   sip_timestamp          datetime64[ns, UTC]
 9   size                   int64              
 10  tape                   int8               
 11  trf_id                 int64              
 12  trf_timestamp          datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), float64(1), int64(3), int8(2), object(4)
memory usage: 3.7+ GB


,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp
0,A,"12,37",0,4,A0031I,2016-01-04 13:39:03.451000+00:00,41.81,1048,2016-01-04 13:39:03.661970+00:00,2,1,12,2016-01-04 13:39:03.661032+00:00
1,A,37,0,12,900]:C,2016-01-04 14:30:01.336263+00:00,41.08,2136,2016-01-04 14:30:01.337114+00:00,87,1,0,2016-01-04 05:00:00+00:00
2,A,16,0,12,900]:D,2016-01-04 14:30:01.336281+00:00,41.08,2137,2016-01-04 14:30:01.337191+00:00,87,1,0,2016-01-04 05:00:00+00:00
3,A,16,0,11,00QJLS,2016-01-04 14:30:07.010978+00:00,41.08,2775,2016-01-04 14:30:07.011305+00:00,35,1,0,2016-01-04 05:00:00+00:00
4,A,"17,37",0,11,00QJLT,2016-01-04 14:30:07.010978+00:00,41.08,2776,2016-01-04 14:30:07.011399+00:00,35,1,0,2016-01-04 05:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43663712,ZYNE,15,0,11,015JG3U,2016-01-04 21:00:00.005669+00:00,9.66,2492444,2016-01-04 21:00:00.068389+00:00,100,3,0,2016-01-04 05:00:00+00:00
43663713,ZYNE,"8,9",0,12,900V5GH,2016-01-04 21:00:00.176919+00:00,9.67,2492709,2016-01-04 21:00:00.177214+00:00,1685,3,0,2016-01-04 05:00:00+00:00
43663714,ZYNE,15,0,12,900V5G^,2016-01-04 21:00:00.177017+00:00,9.67,2492711,2016-01-04 21:00:00.177255+00:00,1685,3,0,2016-01-04 05:00:00+00:00
43663715,ZYNE,"12,37",0,4,009Y9U8,2016-01-04 21:28:37.755000+00:00,9.67,2499903,2016-01-04 21:28:39.775379+00:00,93,3,12,2016-01-04 21:28:39.775003+00:00


In [18]:
df['correction'].value_counts()

correction
0     43663101
8          256
10         256
1           52
12          52
Name: count, dtype: int64

In [15]:
df['conditions'].value_counts()
show(_)

In [16]:
%%time
fragments = trades_ds.get_fragments()
for fragment in fragments:
    table = fragment.to_table(schema=trades_ds.schema)
    table = table.cast(real_schema)
    print(f"{table.num_rows=}")

table.num_rows=43663717
table.num_rows=36107748
table.num_rows=40086971
CPU times: user 43.8 s, sys: 3.2 s, total: 47 s
Wall time: 25.6 s


In [134]:
pa.scalar(0, type=pa.timestamp('ns', tz='UTC'))

<pyarrow.TimestampScalar: '1970-01-01T00:00:00.000000000+0000'>

In [135]:
pa.scalar(1063200649800000000, type=pa.timestamp('ns', tz='UTC'))

<pyarrow.TimestampScalar: '2003-09-10T13:30:49.800000000+0000'>

In [136]:
trades_ds.head(10)

pyarrow.Table
ticker: string not null
conditions: string not null
correction: int8 not null
exchange: int8 not null
id: string not null
participant_timestamp: int64
price: double not null
sequence_number: int64
sip_timestamp: int64 not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: int64
----
ticker: [["A","A","A","A","A","A","A","A","A","A"]]
conditions: [["","","","","","","","","",""]]
correction: [[0,0,0,0,0,0,0,0,0,0]]
exchange: [[10,9,9,9,9,9,9,9,9,9]]
id: [["","","","","","","","","",""]]
participant_timestamp: [[0,0,0,0,0,0,0,0,0,0]]
price: [[25.4,25.4,25.4,25.4,25.4,25.4,25.4,25.4,25.4,25.4]]
sequence_number: [[1929946249800780,1929946251320893,1929946251325765,1929946251330702,1929946251334042,1929946251337323,1929946251377613,1929946251381171,1929946251384456,1929946251392026]]
sip_timestamp: [[1063200649800000000,1063200651320000000,1063200651325000000,1063200651330000000,1063200651334000000,1063200651337000000,1063200651377000000,106320

In [137]:
batches = trades_ds.to_batches()
batch = next(batches)
batch = batch.cast(real_schema)
df = batch.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17177 entries, 0 to 17176
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   ticker                 17177 non-null  object             
 1   conditions             17177 non-null  object             
 2   correction             17177 non-null  int8               
 3   exchange               17177 non-null  int8               
 4   id                     17177 non-null  object             
 5   participant_timestamp  17177 non-null  datetime64[ns, UTC]
 6   price                  17177 non-null  float64            
 7   sequence_number        17177 non-null  int64              
 8   sip_timestamp          17177 non-null  datetime64[ns, UTC]
 9   size                   17177 non-null  int64              
 10  tape                   17177 non-null  int8               
 11  trf_id                 17177 non-null  int64          

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp
0,A,,0,10,,1970-01-01 00:00:00+00:00,25.40,1929946249800780,2003-09-10 13:30:49.800000+00:00,42700,1,0,1970-01-01 00:00:00+00:00
1,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251320893,2003-09-10 13:30:51.320000+00:00,500,1,0,1970-01-01 00:00:00+00:00
2,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251325765,2003-09-10 13:30:51.325000+00:00,500,1,0,1970-01-01 00:00:00+00:00
3,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251330702,2003-09-10 13:30:51.330000+00:00,500,1,0,1970-01-01 00:00:00+00:00
4,A,,0,9,,1970-01-01 00:00:00+00:00,25.40,1929946251334042,2003-09-10 13:30:51.334000+00:00,200,1,0,1970-01-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,AAPL,,0,4,,1970-01-01 00:00:00+00:00,22.29,1361095,2003-09-10 18:56:49+00:00,300,3,0,1970-01-01 00:00:00+00:00
17173,AAPL,,0,3,,1970-01-01 00:00:00+00:00,22.28,1361195,2003-09-10 18:56:51+00:00,100,3,0,1970-01-01 00:00:00+00:00
17174,AAPL,,0,12,,1970-01-01 00:00:00+00:00,22.29,1361445,2003-09-10 18:56:56+00:00,100,3,0,1970-01-01 00:00:00+00:00
17175,AAPL,,0,12,,1970-01-01 00:00:00+00:00,22.29,1361467,2003-09-10 18:56:56+00:00,200,3,0,1970-01-01 00:00:00+00:00


In [45]:
test_csv_path = Path(config.trades_dir) / "2020/10/2020-10-08.csv.gz"
# 2003/09/2003-09-10
test_csv_path = Path(config.trades_dir) / "2003/09/2003-09-10.csv.gz"
print(f"{test_csv_path.exists()=}")
test_csv_path

test_csv_path.exists()=True


PosixPath('/media/mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2003/09/2003-09-10.csv.gz')

In [46]:
table = pa_csv.read_csv(test_csv_path)
df = table.to_pandas()
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp
0,A,,0,10,None,0,25.40,1929946249800780,1063200649800000000,42700,1,0,0
1,A,,0,9,None,0,25.40,1929946251320893,1063200651320000000,500,1,0,0
2,A,,0,9,None,0,25.40,1929946251325765,1063200651325000000,500,1,0,0
3,A,,0,9,None,0,25.40,1929946251330702,1063200651330000000,500,1,0,0
4,A,,0,9,None,0,25.40,1929946251334042,1063200651334000000,200,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5608278,ZXZZT,12,0,12,None,0,817.31,1786105,1063224060000000000,900,3,0,0
5608279,ZXZZT,15,0,12,None,0,565.00,1788231,1063224093000000000,0,3,0,0
5608280,ZXZZT,15,0,3,None,0,568.08,1791346,1063224185000000000,0,3,0,0
5608281,ZXZZT,12,0,11,None,0,400.00,1801735,1063229242000000000,100,3,0,0


https://polygon.io/blog/understanding-trade-eligibility

```
The UTP Sale condition matrix is available on our Conditions and Indicators page. This spec answers most questions asked about the eligibility for trades to update the volume value.

Similar to the CTA matrix, this table outlines the eligibility of each value to be updated based on two sets of guidelines: the Consolidated Processing Guidelines and the Market Center Processing Guidelines.

The Consolidated Processing Guidelines pertain to the consolidated tape, specifically the UTP SIP feed.
The Market Center Processing Guidelines apply to individual participants, such as exchanges.
```

In [109]:
df['conditions'].value_counts()

conditions
         5279655
3         231358
12         56613
2          19406
32          9276
15          6280
22          5077
20           303
4             91
7             85
17            72
29            51
30            10
5              4
32,3           1
20,30          1
Name: count, dtype: int64

In [110]:
df['correction'].value_counts()

correction
0     5603141
8        1465
10       1465
1        1025
12       1025
7          81
11         81
Name: count, dtype: int64

In [111]:
df['exchange'].value_counts()

exchange
12    1863008
10    1464468
11     959078
3      816373
4      319600
9       79566
2       53415
1       41959
17      10749
16         67
Name: count, dtype: int64

In [112]:
df['id'].value_counts()

id
    5608283
Name: count, dtype: int64

In [113]:
df['trf_id'].value_counts()

trf_id
0    5608283
Name: count, dtype: int64

In [160]:
fragment = next(fragments)
table = fragment.to_table(schema=trades_ds.schema)
table = table.cast(real_schema)
df = table.to_pandas()
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788834 entries, 0 to 4788833
Data columns (total 13 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   ticker                 object             
 1   conditions             object             
 2   correction             int8               
 3   exchange               int8               
 4   id                     object             
 5   participant_timestamp  datetime64[ns, UTC]
 6   price                  float64            
 7   sequence_number        int64              
 8   sip_timestamp          datetime64[ns, UTC]
 9   size                   int64              
 10  tape                   int8               
 11  trf_id                 int64              
 12  trf_timestamp          datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), float64(1), int64(3), int8(3), object(3)
memory usage: 379.1+ MB


,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp
0,A,,0,10,,1970-01-01 00:00:00+00:00,25.04,1930119140686194,2003-09-12 13:32:20.686000+00:00,47600,1,0,1970-01-01 00:00:00+00:00
1,A,,0,9,,1970-01-01 00:00:00+00:00,25.04,1930119141687554,2003-09-12 13:32:21.687000+00:00,2500,1,0,1970-01-01 00:00:00+00:00
2,A,,0,9,,1970-01-01 00:00:00+00:00,25.04,1930119141729879,2003-09-12 13:32:21.729000+00:00,100,1,0,1970-01-01 00:00:00+00:00
3,A,,0,12,,1970-01-01 00:00:00+00:00,25.04,1930119142186214,2003-09-12 13:32:22.186000+00:00,200,1,0,1970-01-01 00:00:00+00:00
4,A,,0,12,,1970-01-01 00:00:00+00:00,25.04,1930119143186683,2003-09-12 13:32:23.186000+00:00,1500,1,0,1970-01-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4788829,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525497,2003-09-12 20:31:45+00:00,100,3,0,1970-01-01 00:00:00+00:00
4788830,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525499,2003-09-12 20:31:47+00:00,100,3,0,1970-01-01 00:00:00+00:00
4788831,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525504,2003-09-12 20:32:13+00:00,100,3,0,1970-01-01 00:00:00+00:00
4788832,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525505,2003-09-12 20:32:15+00:00,100,3,0,1970-01-01 00:00:00+00:00


In [161]:
pd.to_datetime(df['sip_timestamp']).dt.date.value_counts()

sip_timestamp
2003-09-12    4788834
Name: count, dtype: int64

In [163]:
pd.to_datetime(df['sip_timestamp']).dt.time.value_counts()

sip_timestamp
13:30:01           1738
13:30:00           1722
19:59:58           1385
20:01:32           1359
19:59:59           1330
                   ... 
19:04:18.713000       1
19:04:16.747000       1
19:04:05.212000       1
19:04:04.713000       1
20:32:18              1
Name: count, Length: 1718912, dtype: int64

In [222]:
def cast_strings_to_list(string_array, separator=",", default="0", value_type=pa.uint8()):
    """Cast a PyArrow StringArray of comma-separated numbers to a ListArray of values."""

    # Create a mask to identify empty strings
    is_empty = pa_compute.equal(pa_compute.utf8_trim_whitespace(string_array), "")

    # Use replace_with_mask to replace empty strings with the default ("0")
    filled_column = pa_compute.replace_with_mask(string_array, is_empty, pa.scalar(default))
    
    # Split the strings by comma
    split_array = pa_compute.split_pattern(filled_column, pattern=separator)

    # Cast each element in the resulting lists to integers
    int_list_array = pa_compute.cast(split_array, pa.list_(value_type))

    return int_list_array

In [223]:
table2 = table.append_column('condition_values', cast_strings_to_list(table.column("conditions").combine_chunks()))
df2 = table2.to_pandas()
df2

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,condition_values
0,A,,0,10,,1970-01-01 00:00:00+00:00,25.04,1930119140686194,2003-09-12 13:32:20.686000+00:00,47600,1,0,1970-01-01 00:00:00+00:00,[0]
1,A,,0,9,,1970-01-01 00:00:00+00:00,25.04,1930119141687554,2003-09-12 13:32:21.687000+00:00,2500,1,0,1970-01-01 00:00:00+00:00,[0]
2,A,,0,9,,1970-01-01 00:00:00+00:00,25.04,1930119141729879,2003-09-12 13:32:21.729000+00:00,100,1,0,1970-01-01 00:00:00+00:00,[0]
3,A,,0,12,,1970-01-01 00:00:00+00:00,25.04,1930119142186214,2003-09-12 13:32:22.186000+00:00,200,1,0,1970-01-01 00:00:00+00:00,[0]
4,A,,0,12,,1970-01-01 00:00:00+00:00,25.04,1930119143186683,2003-09-12 13:32:23.186000+00:00,1500,1,0,1970-01-01 00:00:00+00:00,[0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4788829,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525497,2003-09-12 20:31:45+00:00,100,3,0,1970-01-01 00:00:00+00:00,[0]
4788830,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525499,2003-09-12 20:31:47+00:00,100,3,0,1970-01-01 00:00:00+00:00,[0]
4788831,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525504,2003-09-12 20:32:13+00:00,100,3,0,1970-01-01 00:00:00+00:00,[0]
4788832,ZXZZT,,0,3,,1970-01-01 00:00:00+00:00,0.99,1525505,2003-09-12 20:32:15+00:00,100,3,0,1970-01-01 00:00:00+00:00,[0]


In [224]:
df2["condition_values"].value_counts()

condition_values
[0]         4489947
[3]          220109
[12]          47406
[2]           11175
[32]           8627
[15]           6244
[22]           4962
[20]            157
[7]              65
[4]              53
[29]             44
[17]             34
[30]             10
[29, 12]          1
Name: count, dtype: int64

In [226]:
table2.schema

ticker: string not null
conditions: string not null
correction: int8 not null
exchange: int8 not null
id: string not null
participant_timestamp: timestamp[ns, tz=UTC] not null
price: double not null
sequence_number: int64 not null
sip_timestamp: timestamp[ns, tz=UTC] not null
size: int64 not null
tape: int8 not null
trf_id: int64 not null
trf_timestamp: timestamp[ns, tz=UTC] not null
condition_values: list<item: uint8>
  child 0, item: uint8

In [ ]:
df['date'] = df['sip_timestamp'].ts.date
df['date'].value_count()

In [10]:
config.start_timestamp.date()

datetime.date(2016, 1, 5)

In [10]:
from polygon import RESTClient

client = RESTClient()  # POLYGON_API_KEY environment variable is used
conditions = []
for c in client.list_conditions(asset_class="stocks", limit=1000):
    conditions.append(c)
conditions

[Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=1, legacy=None, name='Acquisition', sip_mapping=SipMapping(CTA=None, OPRA=None, UTP='A'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True))),
 Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=2, legacy=None, name='Average Price Trade', sip_mapping=SipMapping(CTA='B', OPRA=None, UTP='W'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True))),
 Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=No

In [11]:
trade_conditions = {condition.id: condition for condition in conditions if "trade" in condition.data_types and not condition.legacy}
trade_conditions

{1: Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=1, legacy=None, name='Acquisition', sip_mapping=SipMapping(CTA=None, OPRA=None, UTP='A'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True))),
 2: Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=2, legacy=None, name='Average Price Trade', sip_mapping=SipMapping(CTA='B', OPRA=None, UTP='W'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True))),
 3: Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, ex

In [12]:
type(trade_conditions)

dict

In [13]:
df.iloc[0]

ticker                                     A
conditions                       14,12,37,41
correction                                 0
exchange                                  11
id                            52983525037648
participant_timestamp    1602155658646874112
price                                 104.12
sequence_number                       295001
sip_timestamp            1602155658646918400
size                                       5
tape                                       1
trf_id                                     0
trf_timestamp                              0
Name: 0, dtype: object

In [14]:
df.iloc[0].conditions

'14,12,37,41'

In [15]:
trade_conditions[14].update_rules.consolidated.updates_open_close

True

In [16]:
%%cython

from polygon import RESTClient

client = RESTClient()  # POLYGON_API_KEY environment variable is used
conditions = []
for c in client.list_conditions(asset_class="stocks", limit=1000):
    conditions.append(c)

# trade_conditions = {condition.id: condition for condition in conditions if "trade" in condition.data_types and not condition.legacy}
# Need legacy conditions because they appear sometimes, like 35 (Stock Option Trade) on 2020-10-08.
trade_conditions = {condition.id: condition for condition in conditions if "trade" in condition.data_types}

updates_open_close_map = {}
updates_high_low_map = {}
updates_volume_map = {}


def updates_open_close(conditions):
    if not conditions in updates_open_close_map:
        should_update = True
        if conditions.strip():
            for id in conditions.split(","):
                condition = trade_conditions[int(id)]
                if condition.type == "sale_condition":
                    if not condition.update_rules.consolidated.updates_open_close:
                        should_update = False
                        break
        # Only update cache once.
        updates_open_close_map[conditions] = should_update
    return updates_open_close_map[conditions]


def updates_high_low(conditions):
    if not conditions in updates_high_low_map:
        should_update = True
        if conditions.strip():
            for id in conditions.split(","):
                condition = trade_conditions[int(id)]
                if condition.type == "sale_condition":
                    if not condition.update_rules.consolidated.updates_high_low:
                        should_update = False
                        break
        # Only update cache once.
        updates_high_low_map[conditions] = should_update
    return updates_high_low_map[conditions]


def updates_volume(conditions):
    if not conditions in updates_volume_map:
        should_update = True
        if conditions.strip():
            for id in conditions.split(","):
                condition = trade_conditions[int(id)]
                if condition.type == "sale_condition":
                    if not condition.update_rules.consolidated.updates_volume:
                        should_update = False
                        break
        # Only update cache once.
        updates_volume_map[conditions] = should_update
    return updates_volume_map[conditions]


def open_close_price(row):
    if row.conditions.strip():
        for id in row.conditions.split(","):
            condition = trade_conditions[int(id)]
            if condition.type == "sale_condition":
                if not condition.update_rules.consolidated.updates_open_close:
                    return None
    return row["price"]

In [17]:
conditions_values = df.conditions.unique()
conditions_values.tolist()

['14,12,37,41',
 '12',
 '7,12,37',
 '12,37',
 '17,41',
 '16',
 '37',
 '14,41',
 '',
 '14,37,41',
 '10,2,41',
 '10,37,41',
 '53,52,41',
 '15',
 '8,41',
 '12,22',
 '38,41',
 '20,12,41',
 '14,12,41',
 '12,41',
 '12,37,41',
 '37,41',
 '41',
 '12,2',
 '10,41',
 '17,9,41',
 '17,37,41',
 '13,37',
 '53,35,41',
 '32,37',
 '32',
 '7,41',
 '7,37,41',
 '53,41',
 '8,9,41',
 '13',
 '20,12,37,41',
 '20,37,41',
 '53,37,41',
 '7,37',
 '2',
 '20,41',
 '13,22',
 '29,41',
 '20,37',
 '8,37,41',
 '15,41',
 '38',
 '20,12,37',
 '16,41',
 '22,41',
 '7,12',
 '32,2',
 '20,12,2',
 '32,37,41',
 '10,22,41',
 '14,2,41',
 '20,12,22',
 '10,32,37,41',
 '32,22',
 '13,2',
 '13,37,41',
 '52',
 '32,41',
 '14',
 '12,2,41',
 '7,12,37,41',
 '53,32,52,41',
 '20,12',
 '20,10,2,41',
 '2,41',
 '7,10,37,41',
 '53,12,41',
 '7,13,37',
 '20,53,35,41',
 '7,10,41',
 '35',
 '7,12,2,41',
 '18,41',
 '12,22,41',
 '14,32,41',
 '20,53,52,41',
 '29,13',
 '20,13,41',
 '22',
 '20,32,41',
 '20,14,41',
 '29,37,41',
 '7,12,41',
 '7,32,37',
 '20,2,

In [27]:
%%time
[updates_open_close(condition)for condition in conditions_values.tolist()]

CPU times: user 76 μs, sys: 16 μs, total: 92 μs
Wall time: 95.1 μs


[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [28]:
%%time
[updates_open_close(condition)for condition in conditions_values.tolist()]

CPU times: user 14 μs, sys: 1e+03 ns, total: 15 μs
Wall time: 15.7 μs


[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [18]:
open_close_price(df.iloc[0])

In [18]:
%%time
df["updates_open_close"] = df.apply(lambda row: updates_open_close(row.conditions), axis=1) 
df["updates_high_low"] = df.apply(lambda row: updates_high_low(row.conditions), axis=1)
df["updates_volume"] = df.apply(lambda row: updates_volume(row.conditions), axis=1)
df

CPU times: user 8min 4s, sys: 21.9 s, total: 8min 26s
Wall time: 8min 26s


,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,updates_open_close,updates_high_low,updates_volume
0,A,"14,12,37,41",0,11,52983525037648,1602155658646874112,104.12,295001,1602155658646918400,5,1,0,0,False,False,True
1,A,"14,12,37,41",0,11,52983525037649,1602155661112589568,104.00,295201,1602155661112630272,37,1,0,0,False,False,True
2,A,12,0,11,52983525037650,1602155661113007104,104.00,295301,1602155661113041920,158,1,0,0,False,False,True
3,A,"7,12,37",0,4,71675223290963,1602156721498000000,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,False,False,True
4,A,"12,37",0,8,52983525172821,1602161883608071000,104.00,771101,1602161883608327936,1,1,0,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,1602196962599455000,18.51,4207423,1602196962599698841,240,3,0,0,False,False,True
48420263,ZYXI,12,0,4,1046,1602199475132000000,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,False,False,True
48420264,ZYXI,"14,12,41",0,11,598,1602200844791914496,19.05,4213446,1602200844792282485,100,3,0,0,False,False,True
48420265,ZYXI,"14,12,41",0,11,599,1602200877544689664,19.05,4213521,1602200877545059263,100,3,0,0,False,False,True


In [19]:
%%time
df["open_close_price"] = df.apply(lambda row: row.price if updates_open_close(row.conditions) else None, axis=1) 
df["high_low_price"] = df.apply(lambda row: row.price if updates_high_low(row.conditions) else None, axis=1)
df["volume"] = df.apply(lambda row: row.size if updates_volume(row.conditions) else None, axis=1)
df

CPU times: user 9min 56s, sys: 22.5 s, total: 10min 18s
Wall time: 10min 18s


,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,open_close_price,high_low_price,volume
0,A,"14,12,37,41",0,11,52983525037648,1602155658646874112,104.12,295001,1602155658646918400,5,1,0,0,NaN,NaN,15.0
1,A,"14,12,37,41",0,11,52983525037649,1602155661112589568,104.00,295201,1602155661112630272,37,1,0,0,NaN,NaN,15.0
2,A,12,0,11,52983525037650,1602155661113007104,104.00,295301,1602155661113041920,158,1,0,0,NaN,NaN,15.0
3,A,"7,12,37",0,4,71675223290963,1602156721498000000,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,NaN,NaN,15.0
4,A,"12,37",0,8,52983525172821,1602161883608071000,104.00,771101,1602161883608327936,1,1,0,0,NaN,NaN,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,1602196962599455000,18.51,4207423,1602196962599698841,240,3,0,0,NaN,NaN,15.0
48420263,ZYXI,12,0,4,1046,1602199475132000000,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,NaN,NaN,15.0
48420264,ZYXI,"14,12,41",0,11,598,1602200844791914496,19.05,4213446,1602200844792282485,100,3,0,0,NaN,NaN,15.0
48420265,ZYXI,"14,12,41",0,11,599,1602200877544689664,19.05,4213521,1602200877545059263,100,3,0,0,NaN,NaN,15.0


In [16]:
df["open_close_price"] = df.apply(open_close_price, axis=1)
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,open_close_price
0,A,"14,12,37,41",0,11,52983525037648,1602155658646874112,104.12,295001,1602155658646918400,5,1,0,0,NaN
1,A,"14,12,37,41",0,11,52983525037649,1602155661112589568,104.00,295201,1602155661112630272,37,1,0,0,NaN
2,A,12,0,11,52983525037650,1602155661113007104,104.00,295301,1602155661113041920,158,1,0,0,NaN
3,A,"7,12,37",0,4,71675223290963,1602156721498000000,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,NaN
4,A,"12,37",0,8,52983525172821,1602161883608071000,104.00,771101,1602161883608327936,1,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,1602196962599455000,18.51,4207423,1602196962599698841,240,3,0,0,NaN
48420263,ZYXI,12,0,4,1046,1602199475132000000,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,NaN
48420264,ZYXI,"14,12,41",0,11,598,1602200844791914496,19.05,4213446,1602200844792282485,100,3,0,0,NaN
48420265,ZYXI,"14,12,41",0,11,599,1602200877544689664,19.05,4213521,1602200877545059263,100,3,0,0,NaN


In [48]:
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,time_interval
0,A,"14,12,37,41",0,11,52983525037648,2020-10-08 11:14:18.646874112+00:00,104.12,295001,1602155658646918400,5,1,0,0,2020-10-08 11:14:00+00:00
1,A,"14,12,37,41",0,11,52983525037649,2020-10-08 11:14:21.112589568+00:00,104.00,295201,1602155661112630272,37,1,0,0,2020-10-08 11:14:00+00:00
2,A,12,0,11,52983525037650,2020-10-08 11:14:21.113007104+00:00,104.00,295301,1602155661113041920,158,1,0,0,2020-10-08 11:14:00+00:00
3,A,"7,12,37",0,4,71675223290963,2020-10-08 11:32:01.498000+00:00,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,2020-10-08 11:32:00+00:00
4,A,"12,37",0,8,52983525172821,2020-10-08 12:58:03.608071+00:00,104.00,771101,1602161883608327936,1,1,0,0,2020-10-08 12:58:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,2020-10-08 22:42:42.599455+00:00,18.51,4207423,1602196962599698841,240,3,0,0,2020-10-08 22:42:00+00:00
48420263,ZYXI,12,0,4,1046,2020-10-08 23:24:35.132000+00:00,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,2020-10-08 23:24:00+00:00
48420264,ZYXI,"14,12,41",0,11,598,2020-10-08 23:47:24.791914496+00:00,19.05,4213446,1602200844792282485,100,3,0,0,2020-10-08 23:47:00+00:00
48420265,ZYXI,"14,12,41",0,11,599,2020-10-08 23:47:57.544689664+00:00,19.05,4213521,1602200877545059263,100,3,0,0,2020-10-08 23:47:00+00:00


In [29]:
for id in df.iloc[0].conditions.split(","):
    print(f"{id=} {trade_conditions[int(id)]=}")

id='14' trade_conditions[int(id)]=Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=14, legacy=None, name='Intermarket Sweep', sip_mapping=SipMapping(CTA='F', OPRA=None, UTP='F'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True)))
id='12' trade_conditions[int(id)]=Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=12, legacy=None, name='Form T/Extended Hours', sip_mapping=SipMapping(CTA='T', OPRA=None, UTP='T'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True)))
id='37' trade_conditions[int(i

In [33]:
set(df["conditions"].values.tolist())

{'',
 '10,2,41',
 '10,22,41',
 '10,32,37,41',
 '10,32,41',
 '10,37,41',
 '10,41',
 '12',
 '12,2',
 '12,2,41',
 '12,22',
 '12,22,41',
 '12,37',
 '12,37,41',
 '12,41',
 '13',
 '13,2',
 '13,2,41',
 '13,22',
 '13,37',
 '13,37,41',
 '13,41',
 '14',
 '14,12,37,41',
 '14,12,41',
 '14,2,41',
 '14,32,41',
 '14,37,41',
 '14,41',
 '15',
 '15,41',
 '16',
 '16,41',
 '17,37,41',
 '17,41',
 '17,9,41',
 '18,41',
 '18,9,41',
 '2',
 '2,41',
 '20,10,2,41',
 '20,10,41',
 '20,12',
 '20,12,2',
 '20,12,22',
 '20,12,37',
 '20,12,37,41',
 '20,12,41',
 '20,13,2',
 '20,13,37,41',
 '20,13,41',
 '20,14,41',
 '20,2,41',
 '20,32,41',
 '20,37',
 '20,37,41',
 '20,41',
 '20,52,41',
 '20,53,35,41',
 '20,53,52,41',
 '22',
 '22,41',
 '29,10,2,41',
 '29,12,41',
 '29,13',
 '29,37,41',
 '29,41',
 '31',
 '32',
 '32,2',
 '32,22',
 '32,37',
 '32,37,41',
 '32,41',
 '35',
 '37',
 '37,41',
 '38',
 '38,41',
 '41',
 '52',
 '53,12,41',
 '53,32,52,41',
 '53,35,41',
 '53,37,41',
 '53,41',
 '53,52,41',
 '7,10,37,41',
 '7,10,41',
 '7,12'

In [35]:
df["conditions"].value_counts()

conditions
            17680822
37          15121404
14,37,41     7204704
14,41        6721614
12,37         377988
              ...   
7,32,37            1
7,12,41            1
12,22,41           1
20,14,41           1
32,2               1
Name: count, Length: 102, dtype: int64


In [22]:
df["participant_timestamp"] = pd.to_datetime(
    df["participant_timestamp"], unit="ns", utc=True
)

In [29]:
df["time_interval"] = df["participant_timestamp"].dt.floor("60min")

In [30]:
%%time
aggs = df.groupby(["ticker", "time_interval"]).agg({
    'open_close_price': ['first', 'last'],  # Open and Close prices
    'high_low_price': ['max', 'min'],  # High and Low prices
    'volume': 'sum',  # Volume as the sum of size
})
aggs

CPU times: user 2.25 s, sys: 528 ms, total: 2.78 s
Wall time: 2.78 s


open_close_price          high_low_price  \
                                            first     last            max   
ticker time_interval                                                        
A      2020-10-08 11:00:00+00:00              NaN      NaN            NaN   
       2020-10-08 12:00:00+00:00              NaN      NaN            NaN   
       2020-10-08 13:00:00+00:00           104.21  103.445         104.39   
       2020-10-08 14:00:00+00:00           103.47  103.440         103.60   
       2020-10-08 15:00:00+00:00           103.46  103.965         104.25   
...                                           ...      ...            ...   
ZYXI   2020-10-08 19:00:00+00:00            19.18   19.130          19.39   
       2020-10-08 20:00:00+00:00            19.10   19.100          19.10   
       2020-10-08 21:00:00+00:00              NaN      NaN            NaN   
       2020-10-08 22:00:00+00:00              NaN      NaN            NaN   
       2020-10-08 23:00:00+00:00              NaN      NaN            NaN   

                                           volume  
                                     min      sum  
ticker time_interval                               
A      2020-10-08 11:00:00+00:00     NaN     60.0  
       2020-10-08 12:00:00+00:00     NaN     15.0  
       2020-10-08 13:00:00+00:00  103.38  10905.0  
       2020-10-08 14:00:00+00:00  103.13  22845.0  
       2020-10-08 15:00:00+00:00  103.38  21000.0  
...                                  ...      ...  
ZYXI   2020-10-08 19:00:00+00:00   19.09  16515.0  
       2020-10-08 20:00:00+00:00   19.10    195.0  
       2020-10-08 21:00:00+00:00     NaN    210.0  
       2020-10-08 22:00:00+00:00     NaN    105.0  
       2020-10-08 23:00:00+00:00     NaN     60.0  

[90600 rows x 5 columns]

In [31]:
show(aggs)

In [36]:
show(df[df['ticker'] == 'AAPL'])

In [34]:
trade_conditions[11]

Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=11, legacy=None, name='Distribution', sip_mapping=SipMapping(CTA=None, OPRA=None, UTP='D'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True)))

In [35]:
trade_conditions[12]

Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=12, legacy=None, name='Form T/Extended Hours', sip_mapping=SipMapping(CTA='T', OPRA=None, UTP='T'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True)))

In [37]:
trade_conditions[37]

Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=37, legacy=None, name='Odd Lot Trade', sip_mapping=SipMapping(CTA='I', OPRA=None, UTP='I'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True)))

In [38]:
trade_conditions[41]

Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=41, legacy=None, name='Trade Thru Exempt', sip_mapping=SipMapping(CTA='1', OPRA=None, UTP='X'), type='trade_thru_exempt', update_rules=None)

In [40]:
show(df[df['correction'] != 0])

In [41]:
df['correction'].value_counts()

correction
0     48419753
8          242
10         242
1           14
12          14
7            1
11           1
Name: count, dtype: int64